In [16]:
#james chartouni
import lightgbm as lgb
import xgboost as xgb
import pandas as pd
import numpy as np
from numpy import inf
from sklearn.decomposition import PCA, KernelPCA, TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, RobustScaler, PolynomialFeatures
from sklearn.metrics import log_loss

In [17]:
training_data = pd.read_csv("cleaned_input/train_consolidated.csv")
training_data = training_data.drop(['msno'], axis=1)

In [18]:
test_data = pd.read_csv("cleaned_input/test_consolidated.csv")
test_data.shape
#test_data = test_data.drop(['msno'], axis=1)

(503220, 134)

In [19]:
#split data 
y = training_data["is_churn"].values
X = training_data.drop(["is_churn"], axis=1).values
#replaced infinite values with zero. MAKE SURE THIS IS the right thing to do 
X[X == -inf] = 0
X[X == inf] = 0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15, shuffle=True)

In [14]:
#logistic regression 
logistic_pipe = make_pipeline(LogisticRegression(solver='liblinear'))
scores = cross_val_score(logistic_pipe, X_train, y_train, cv=5, scoring="neg_log_loss")
print("Accuracy: %0.2f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

#logistic regression w/ polynomial features 2   
logistic_pipe = make_pipeline(PolynomialFeatures(degree=2), LogisticRegression(solver='liblinear'))
scores = cross_val_score(logistic_pipe, X_train, y_train, cv=5, scoring="neg_log_loss")
print("Accuracy: %0.2f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

#logistic regression w/th PCA, Polynomial features 
logistic_pipe = make_pipeline(PCA(n_components=.95), PolynomialFeatures(degree=2), LogisticRegression(solver='liblinear'))
scores = cross_val_score(logistic_pipe, X_train, y_train, cv=5, scoring="neg_log_loss")
print("Accuracy: %0.2f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))


Accuracy: -0.19 (+/- 0.1852)


/Users/James/anaconda/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


Accuracy: -0.56 (+/- 0.5516)
Accuracy: -0.64 (+/- 0.2019)


/Users/James/anaconda/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [22]:
#Light GBM 
# create dataset for lightgbm
# if you want to re-use data, remember to set free_raw_data=False
lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train,free_raw_data=False)


# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 31,
    'learning_rate': 0.01,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'num_threads': 3
}
# generate a feature name
feature_name = X_train.dtype.names

#cross validation
num_round = 150
scores = lgb.cv(params, lgb_train, num_round, nfold=5, verbose_eval=True)

[1]	cv_agg's binary_logloss: 0.683204 + 3.53977e-07
[2]	cv_agg's binary_logloss: 0.673457 + 6.52159e-07
[3]	cv_agg's binary_logloss: 0.665009 + 2.91504e-06
[4]	cv_agg's binary_logloss: 0.656702 + 6.18411e-06
[5]	cv_agg's binary_logloss: 0.64747 + 5.91019e-06
[6]	cv_agg's binary_logloss: 0.639666 + 8.06596e-06
[7]	cv_agg's binary_logloss: 0.630758 + 7.56281e-06
[8]	cv_agg's binary_logloss: 0.622018 + 7.01405e-06
[9]	cv_agg's binary_logloss: 0.614458 + 8.86047e-06
[10]	cv_agg's binary_logloss: 0.60602 + 8.3214e-06
[11]	cv_agg's binary_logloss: 0.597735 + 8.19125e-06
[12]	cv_agg's binary_logloss: 0.589602 + 8.0337e-06
[13]	cv_agg's binary_logloss: 0.581615 + 7.95395e-06
[14]	cv_agg's binary_logloss: 0.57377 + 7.50067e-06
[15]	cv_agg's binary_logloss: 0.566065 + 7.53806e-06
[16]	cv_agg's binary_logloss: 0.558496 + 7.40981e-06
[17]	cv_agg's binary_logloss: 0.551059 + 7.24943e-06
[18]	cv_agg's binary_logloss: 0.543751 + 7.15322e-06
[19]	cv_agg's binary_logloss: 0.536568 + 7.25076e-06
[20]	cv

In [ ]:
feature_name


In [ ]:
print('Start training...')
# feature_name and categorical_feature
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=50,
                valid_sets=lgb_train,  # eval training data
              )

In [ ]:
print("--------------------------------------------------------------")
ypred = gbm.predict(X_test)
print(log_loss(y_test, ypred))
# save model to file
gbm.save_model('model.txt')

# feature names
print('Feature names:', gbm.feature_name())

# feature importances
print('Feature importances:', list(gbm.feature_importance()))

In [23]:
from sklearn import metrics

def xgb_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'log_loss', metrics.log_loss(labels, preds)

fold = 1
for i in range(fold):
    params = {
        'eta': 0.02, #use 0.002
        'max_depth': 7,
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'seed': i,
        'silent': False
    }
    watchlist = [(xgb.DMatrix(X_train, y_train), 'train'), (xgb.DMatrix(X_test, y_test), 'valid')]
    model = xgb.train(params, xgb.DMatrix(X_train, y_train), 150,  watchlist, feval=xgb_score, maximize=False, verbose_eval=50, early_stopping_rounds=50) #use 1500
    if i != 0:
        pred += model.predict(xgb.DMatrix(test[cols]), ntree_limit=model.best_ntree_limit)
    else:
        pred = model.predict(xgb.DMatrix(test[cols]), ntree_limit=model.best_ntree_limit)
pred /= fold
test['is_churn'] = pred.clip(0.0000001, 0.999999)
test[['msno','is_churn']].to_csv('submission3.csv.gz', index=False, compression='gzip')

[0]	train-logloss:0.673348	valid-logloss:0.673348	train-log_loss:0.673348	valid-log_loss:0.673348
Multiple eval metrics have been passed: 'valid-log_loss' will be used for early stopping.

Will train until valid-log_loss hasn't improved in 50 rounds.
[50]	train-logloss:0.197725	valid-logloss:0.197725	train-log_loss:0.197725	valid-log_loss:0.197725
[100]	train-logloss:0.06819	valid-logloss:0.06819	train-log_loss:0.06819	valid-log_loss:0.06819
[149]	train-logloss:0.025042	valid-logloss:0.025042	train-log_loss:0.025042	valid-log_loss:0.025042


NameError: name 'test' is not defined